# Regression
After a first look at simple linear regression, we want to look at more complex details, potential issues, and how to resolve them. In order to focus on certain data science topics, we will use syntethic data for this part.

## Preparations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import sklearn as sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression, Lasso, lasso_path
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV

## Generate Data
We define the following function to generate a data set (with `n_samples` many entries). As in the previous notebooks, this function returns a DataFrame, here with the attributes `x` and `y`. We will consider `x` as the independent variable and `y` as the dependent variable throughout the notebook.

The function uses a random generator to generate random values ​​for the `x` values ​​and to introduce noise (e.g. a measurement error) on the `y` values.

In [ ]:
def generate_dataset_df(n_samples = 20):
    x = - 4 * np.random.uniform(-1, 1, n_samples)
    y = x - 2 * (x ** 2) + 0.5 * (x ** 3) + np.random.normal(-3, 3, n_samples)
    
    dataset_df = pd.DataFrame({ 'x': x, 'y': y})
    return dataset_df

In [ ]:
generate_dataset_df(n_samples = 5)

We now use this function to generate our dataset, which we will split into a training and test data set. We could of course also generate two separate datasets for training and testing separately, but we'll follow this appraoch to see how to split a single given dataset into a training and test set.

As mentioned, the function uses a random generator. This random generator can be initialized with a specific value - the so-called seed. **You should get always set a seed so that the results are reproducible**, i.e. you can run the notebook again later and get the same results. The actual value, however, does not really matter.

In [ ]:
np.random.seed(0)

In [ ]:
all_data = generate_dataset_df(n_samples=40)

In [ ]:
# get a random sample of the data for training
data_train = all_data.sample(frac=0.5, random_state=1)
# gets the left out portion of the dataset
data_test=all_data.drop(data_train.index)

In [ ]:
data_train

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], data_train['y'], s=10)
plt.grid()
plt.show()

## Linear Regression
Again, we define a Regression model. One way to do so is the class `LinearRegression()` from `sklearn.linear_model`, which we have imported above:

In [ ]:
linreg_deg1 = LinearRegression()
linreg_deg1.fit(data_train[['x']], data_train['y'])

print('Coefficient: ')
print(linreg_deg1.coef_)
print('Intercept:')
print(linreg_deg1.intercept_)

Let's plot the training data along with the fitted model:

In [ ]:
data_forPlot = pd.DataFrame(np.linspace(-4, 4, num=41), columns = ['x'])
data_forPlot['y_lm_grad01'] = linreg_deg1.predict(data_forPlot[['x']])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], data_train['y'], 
            label='Training Data', color='b', s=10)
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad01'],
         label = 'Linear Modell',
         linestyle="--", color='r')

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('y')

plt.grid()
plt.show()

### Calculate & Visualize Residuals
We define a function to get the prediction from a fitted model and compute the residuum values:

In [ ]:
def get_prediction_and_residuum(fitted_model, data_set, y_true):
    y_pred = fitted_model.predict(data_set)
    df_prediction = data_set.copy()
    df_prediction['Prediction'] = y_pred
    df_prediction['Residual'] = y_true - df_prediction['Prediction']
    print('r2-Score: ' + str(r2_score(y_true, y_pred)))
    print('MSE: ' + str(mean_squared_error(y_true, y_pred)))
    print('RMSE: ' + str(root_mean_squared_error(y_true, y_pred)))
    return df_prediction

In [ ]:
lm_grad01_pred_res_train = get_prediction_and_residuum(linreg_deg1,
                                                       data_train[['x']],
                                                       data_train['y'])

We make a plot comparing the predictor values (on the x axis) with the residuum values:

In [ ]:
lm_grad01_pred_res_train

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], lm_grad01_pred_res_train['Residual'],
            label='Training Data', s=10)
plt.legend()

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('Residual ($y-\hat{y}$) of Linear Regression')

plt.plot(plt.xlim(), [0, 0], linestyle="--", color='k')
plt.grid()
plt.show()

We can see a clear structure in these values - for lower values of x, the residuum is alwys negative, in the medium range it is always positive, and for higher values the residuum is negative again. To capture this nonlinear behavior, we define a more complex, non-linear model (in the original values) by defining a regression including the quadratic values of `x`:

## Quadratic Regression
We first do the feature transformation by defining a further feature `x2` as the square of `x`:

In [ ]:
data_train['x2'] = data_train['x']**2

In [ ]:
linreg_deg2_1 = LinearRegression()
linreg_deg2_1.fit(data_train[['x', 'x2']], data_train['y'])

print('Coefficient: ')
print(linreg_deg2_1.coef_)
print('Intercept:')
print(linreg_deg2_1.intercept_)

### Data transformation
The `PolynomialFeatures` from `scikit-learn` uses arrays for the independent and dependent variables - and not DataFrames, as with `statmodels`. We therefore first store the independent variable `x` and the dependent variable `y` as an array, both for the training and test data.

In [ ]:
train_x = data_train[['x']]
train_y = data_train[['y']]

test_x = data_test[['x']]
test_y = data_test[['y']]

Now we can produce the polynomial features. We also convert them into a DataFrame, so that we have the same structure as above.

In [ ]:
polynomial_features02 = PolynomialFeatures(degree=2)
train_x_poly02 = polynomial_features02.fit_transform(train_x)
train_x_poly02_df = pd.DataFrame(train_x_poly02, columns=['x0', 'x1', 'x2'])

We define a new model `model02` for the quadratic regression:

In [ ]:
model02 = LinearRegression()
model02.fit(train_x_poly02_df, train_y)

In [ ]:
print('Coefficient: ')
print(model02.coef_)
print('Intercept:')
print(model02.intercept_)

As we will be using this functionality over and over again, we pack it into a function:

In [ ]:
def print_coeff_and_intercept(model):
    print("Model Coefficients: ")
    print(model.coef_)
    print("Intercept: ")
    print(model.intercept_)

In [ ]:
print_coeff_and_intercept(model02)

In [ ]:
lm_grad02_pred_res_train = get_prediction_and_residuum(model02,
                                                       train_x_poly02_df,
                                                       data_train['y'].reset_index()['y'])

Again, we prepare a lot of the training data and the model:

In [ ]:
data_forPlot_2 = polynomial_features02.fit_transform(data_forPlot[['x']])
data_forPlot_2_df = pd.DataFrame(data_forPlot_2, columns=['x0', 'x1', 'x2'])
data_forPlot['y_lm_grad02'] = model02.predict(data_forPlot_2_df)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], data_train['y'], label='Training Data', color='b', s=10)
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad01'], label = 'Linear Model', linestyle="--", color='r')
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad02'], label = 'Quadratic Model', linestyle="--", color='m')

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('y')

plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], lm_grad02_pred_res_train['Residual'], label='Training Data', s=10)
plt.legend()

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('Residual ($y-\hat{y}$) of Quadratic Regression')

plt.plot(plt.xlim(), [0, 0], linestyle="--", color='k')
plt.grid()
plt.show()

## Qubic Regression
As before, we do the feature transformation by defining a further feature and then fit a linear regression with OLS (ordinary least squares):

**EXERCISE:** Analoguous to the quadratic regression above, do a regression with a polynomial of degree 3. Train and evaluate it.

In [ ]:
# polynomial_features03 = ...
# ...

Again we plot the models and the data:

In [ ]:
data_forPlot_3 = polynomial_features03.fit_transform(data_forPlot[['x']])
data_forPlot_3_df = pd.DataFrame(data_forPlot_3, columns=['x0', 'x1', 'x2', 'x3'])
data_forPlot['y_lm_grad03'] = model03.predict(data_forPlot_3_df)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], data_train['y'], 
            label='Training Data', color='k', s=10)
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad01'], label = 'Linear Model', linestyle="--", color='r')
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad02'], label = 'Quadratic Model', linestyle="--", color='m')
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad03'], label = 'Qubic Model', linestyle="--", color='b')

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('y')

plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], lm_grad03_pred_res_train['Residual'], label='Training Data', s=10)
plt.legend()

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('Residual ($y-\hat{y}$) of Qubic Regression')

plt.plot(plt.xlim(), [0, 0], linestyle="--", color='k')
plt.grid()
plt.show()

## Polynomial Regression, Degree 4:

In [ ]:
polynomial_features04 = PolynomialFeatures(degree=4)
train_x_poly04 = polynomial_features04.fit_transform(train_x)
train_x_poly04_df = pd.DataFrame(train_x_poly04, columns=['x0', 'x1', 'x2', 'x3', 'x4'])

In [ ]:
model04 = LinearRegression()
model04.fit(train_x_poly04_df, train_y)

In [ ]:
print_coeff_and_intercept(model04)

In [ ]:
data_forPlot_4 = polynomial_features04.fit_transform(data_forPlot[['x']])
data_forPlot_4_df = pd.DataFrame(data_forPlot_4, columns=['x0', 'x1', 'x2', 'x3', 'x4'])
data_forPlot['y_lm_grad04'] = model04.predict(data_forPlot_4_df)

In [ ]:
lm_grad04_pred_res_train = get_prediction_and_residuum(model04,
                                                       train_x_poly04_df,
                                                       data_train['y'].reset_index()['y'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], data_train['y'], 
            label='Training Data', color='k', s=10)
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad01'], label = 'Linear Model', linestyle="--", color='r')
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad02'], label = 'Quadratic Model', linestyle="--", color='m')
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad03'], label = 'Qubic Model', linestyle="--", color='b')
plt.plot(data_forPlot['x'], data_forPlot['y_lm_grad04'], label = '4th-Order Model', linestyle="--", color='g')

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('y')

plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plt.scatter(data_train['x'], lm_grad04_pred_res_train['Residual'], label='Training Data', s=10)
plt.legend()

plt.legend()
ax.set_xlabel('x')
ax.set_ylabel('Residual ($y-\hat{y}$) of 4th-Order Regression')

plt.plot(plt.xlim(), [0, 0], linestyle="--", color='k')
plt.grid()
plt.show()

The difference to the qubic regression (both with respect to the model predictions and the residui.

## Polynomial Regression with Degree 20

**EXERCISE:** Analoguous to the examples above, do a regression with a polynomial of degree 20. Train and evaluate it. What do you think about it?

In [ ]:
# prepare data
# polynomial_features20 = ...
# train_x_poly20 = ...
# train_x_poly20_df = ...

For a better understanding, let's look at the parameters of the model, and compare them with some parameters we got for models of lower degree:

In [ ]:
print_coeff_and_intercept(model20)

In [ ]:
print_coeff_and_intercept(model02)

The parameters in `model20` are much larger. In addition, the degree of the polynomial is high! $x^{20}$ grows incredibly quickly!

In [ ]:
np.array([0.0, 1.0, 2.0, 3.0, 4.0])**20

$4^{20}$ is already over a trillion (one million million). At the same time, ${(1/4)}^{20}$ is less than a trillionth. With such different values, the computer reaches its limits, resulting in so-called *numerical instabilities*. Incidentally, these are also the reason why the regression does not go through all the points exactly.

## Learning as Generalization: Systematic Performance Comparison on Training and Test Data
We now want to look at the performance of models with different degrees on the training and test data. First, we sketch our program in pseudo-code:

In [ ]:
# ...

We also write an output function for the root mean square error and the R-squared coefficient:

In [ ]:
def get_rmse_r2(true_y, pred_y, doPrint=False):
    rmse = np.sqrt(mean_squared_error(true_y, pred_y))
    r2 = r2_score(true_y, pred_y)
    
    if doPrint:
        print("Mean Squared Error (RMSE): " + str(rmse))
        print("R-squared: " + str(r2))
        
    return [rmse, r2]

Now we are ready to implement the systematic performance comparison:

In [ ]:
# prepare arrays to store performance results
train_performance = np.array([])
test_performance = np.array([])
max_degree = 10

# for different degrees of model - from degree 1 to 10
for degree in range(0, max_degree):

    # Prepare data: e.g. polynomial features of degree 7
    polynomial_features = PolynomialFeatures(degree=degree)
    train_x_poly = polynomial_features.fit_transform(train_x)
    
    # Define model of selected degree
    model = LinearRegression()

    # Train model with training data
    model.fit(train_x_poly, train_y)

    # Measure R-squared and RMSE (root mean squared error)
    # - on training data:
    train_y_pred = model.predict(train_x_poly)
    train_performance = np.append(train_performance, 
                                  get_rmse_r2(train_y, train_y_pred),
                                  axis=0)
    
    #  Measure R-squared and RMSE (root mean squared error)
    # - on test data:
    test_x_poly = polynomial_features.fit_transform(test_x)
    test_y_pred = model.predict(test_x_poly)
    test_performance = np.append(test_performance,
                                 get_rmse_r2(test_y, test_y_pred),
                                 axis=0)
    
train_performance = train_performance.reshape(-1, 2)
test_performance = test_performance.reshape(-1, 2)

With `append(test_performance, get_rmse_r2(test_y, test_y_pred), axis=0)` the new results that we receive via the function `get_rmse_r2(test_y, test_y_pred)` are appended to the existing array.

Finally, it is transformed into a two-dimensional structure with `train_performance = train_performance.reshape(-1, 2)` and `test_performance = test_performance.reshape(-1, 2)`.

Next, we create a graph to plot the mean square error or R-score as a function of the polynomial degree:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))

ax[0].plot(range(0, max_degree), train_performance[:, 0], color='b',
           label='Training Data')
ax[0].plot(range(0, max_degree), test_performance[:, 0], color='g',
           label='Test Data')
ax[0].grid()
ax[0].set_title('RMSE')
ax[0].set_xlabel('Degree of Polynomial')

ax[1].plot(range(0, max_degree), train_performance[:, 1], color='b',
           label='Training Data')
ax[1].plot(range(0, max_degree), test_performance[:, 1], color='g',
           label='Test Data')
ax[1].grid()
ax[1].set_title('R-Squared')
ax[1].set_xlabel('Degree of Polynomial')

plt.legend()
plt.show()

We can thus adapt the model complexity to the test data. With a polynomial of degree 4 we achieve the best quality in terms of both quality measures (RMSE and R-Squared). The appropriate degree for the polynomial regression is therefore 4.

**BUT**: we have now determined the model complexity using the test data... and we may also be overfitting! So we need a third, separate data set to determine the final quality.

### Crossvalidation
We now use crossvalidation to first determine the model complexity and then - for the model of the selected complexity - to be able to make a final statement about the quality of the regression.

In the `sklean` library there is a function `cross_validate` which handles the crossvalidation. The function requires
* a model that implements the functions `fit()` and `predict`
* the independent variables of a training data set
* the dependent variables of the training data set
* `cv`, the number of runs
* `scoring`, the quality measures to be used

Details of the function can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html
Note that the quality measures are always chosen according to convention so that more is better. R-Square has this property and is offered as `'explained_variance'`. With RMSE, however, less is better; here the negative RMSE is therefore implemented as `'neg_root_mean_squared_error'` as a quality measure. Details: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
polynomial_features = PolynomialFeatures(degree=3)
train_x_poly = polynomial_features.fit_transform(train_x)

model = LinearRegression()
cv_result = cross_validate(model, train_x_poly, train_y, cv=5, 
                           scoring=('explained_variance', 'neg_root_mean_squared_error'),
                           return_train_score=True)

The resulting `cv_result` contains the metrics for every one of the cross-validation folds - i.e., 5 values in our case:

In [ ]:
cv_result

Technically, the result `cv_result` is a dictionary, where each value is an array with the respective measurements for every cross-validation fold. Hence, we can for example compute the mean and standard deviation over all cross-validation folds:

In [ ]:
for name, valarr in cv_result.items():
    print(name + ": " + '{:.4}'.format(np.mean(valarr)) + "+-" + '{:.4}'.format(np.std(valarr)))

### Running Cross-Validation for Varying Polynom Degree
Next, we run the cross-validation for polynomial regression models with varying degree. We do this once manually, and will then see how we can use predefined Python functionality to do so:

In [ ]:
train_performance_rms_mean = np.zeros(max_degree)
train_performance_rms_mean

In [ ]:
overall_train_x = train_x
overall_train_y = train_y

In [ ]:
# Initialise arrays
max_degree = 10

train_performance_rms_mean = np.zeros(max_degree+1)
train_performance_r2_mean = np.zeros(max_degree+1)
train_performance_rms_std = np.zeros(max_degree+1)
train_performance_r2_std = np.zeros(max_degree+1)

val_performance_rms_mean = np.zeros(max_degree+1)
val_performance_r2_mean = np.zeros(max_degree+1)
val_performance_rms_std = np.zeros(max_degree+1)
val_performance_r2_std = np.zeros(max_degree+1)

# for-loop:
for degree in range(0, max_degree+1):
    # genearte polynomial features 
    polynomial_features = PolynomialFeatures(degree=degree)
    overall_train_x_poly = polynomial_features.fit_transform(overall_train_x)
    
    # define model and train via cross-validation
    model = LinearRegression()
    cv_result = cross_validate(model, overall_train_x_poly, overall_train_y, cv=5,
                               scoring=('explained_variance', 'neg_root_mean_squared_error'),
                               return_train_score=True)

    # summarize evaluation on training data
    train_performance_rms_mean[degree] = -np.mean(cv_result['train_neg_root_mean_squared_error'])
    train_performance_rms_std[degree]  = np.std(-cv_result['train_neg_root_mean_squared_error'])

    train_performance_r2_mean[degree] = np.mean(cv_result['train_explained_variance'])
    train_performance_r2_std[degree]  = np.std(cv_result['train_explained_variance'])

    # summarize evaluation on validation data
    val_performance_rms_mean[degree] = -np.mean(cv_result['test_neg_root_mean_squared_error'])
    val_performance_rms_std[degree]  =  np.std(-cv_result['test_neg_root_mean_squared_error'])

    val_performance_r2_mean[degree] = np.mean(cv_result['test_explained_variance'])
    val_performance_r2_std[degree]  = np.std(cv_result['test_explained_variance'])

Let's again plot the performance metrics versus the degree of the polynomial:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,6))

ax[0].plot(range(0, max_degree+1), train_performance_rms_mean, color='b', label='Training Data')
ax[0].plot(range(0, max_degree+1), val_performance_rms_mean, color='g', label='Validation Data')
ax[0].set_title('RMSE')
ax[0].set_ylim(0, 15)
ax[0].grid()
ax[0].set_xlabel('Degree of Polynomial')


ax[1].plot(range(0, max_degree+1), train_performance_r2_mean, color='b', label='Training Data')
ax[1].plot(range(0, max_degree+1), val_performance_r2_mean, color='g', label='Validation Data')
ax[1].set_title('R-Squared')
ax[1].set_ylim(0, 1)
ax[1].grid()
ax[1].set_xlabel('Degree of Polynomial')

plt.legend()
plt.show()

Note that the curves are now different from the ones we obtained above. This is because we have split the training data used above into training and validation data. Hence, from the original 20 data in `train_x` used above, we now only use 80% for training, and the remaining 20% for evaluation. Hence, the training data consists of only 16 samples.

We extend the plot to include the standard deviation of the performance metrics over the 5 cross-validation sets:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,6))

train_performance_rms_mean = np.array(train_performance_rms_mean)
train_performance_rms_std = np.array(train_performance_rms_std)

ax[0].plot(range(0, max_degree+1), train_performance_rms_mean, color='b', label='Training Data')
ax[0].fill_between(range(0, max_degree+1), train_performance_rms_mean-1.96*train_performance_rms_std,
                         train_performance_rms_mean+1.96*train_performance_rms_std, color='b', alpha=.15)
ax[0].plot(range(0, max_degree+1), val_performance_rms_mean, color='g', label='Validation Data')
ax[0].fill_between(range(0, max_degree+1), val_performance_rms_mean-1.96*val_performance_rms_std,
                         val_performance_rms_mean+1.96*val_performance_rms_std, color='g', alpha=.15)

ax[0].set_title('RMSE')
ax[0].set_ylim(0, 15)
ax[0].grid()
ax[0].set_xlabel('Degree of Polynomial')

ax[1].plot(range(0, max_degree+1), train_performance_r2_mean, color='b', label='Training Data')
ax[1].fill_between(range(0, max_degree+1), train_performance_r2_mean-1.96*train_performance_r2_std,
                         train_performance_r2_mean+1.96*train_performance_r2_std, color='b', alpha=.15)
ax[1].plot(range(0, max_degree+1), val_performance_r2_mean, color='g', label='Validation Data')
ax[1].fill_between(range(0, max_degree+1), val_performance_r2_mean-1.96*val_performance_r2_std,
                         val_performance_r2_mean+1.96*val_performance_r2_std, color='g', alpha=.15)

ax[1].set_title('R-Squared')
ax[1].set_ylim(-10, 5)
ax[1].grid()
ax[1].set_xlabel('Degree of Polynomial')

plt.legend()
plt.show()

## Grid Search
As hyperparameter optimization is a very common task, the `scikit-learn` framework offers the `GridSearchCV` class to systematically evaluate a grid of hyperparameters using crossvalidation in order to find the best set of hyperparameters. To make use of the function, one typically defines a so-called `Pipeline`, which comprises all steps including e.g. preprocessing. Also, a grid of all hyperparameters to be evaluated has to be passed; the `fit` function will then determine the best hyperparameter tuple based on a training data set, doing crossvalidation internally.

In [ ]:
from sklearn.pipeline import Pipeline
def PolynomialRegression(degree=2, **kwargs):
    return Pipeline(steps=[('polyfeat', PolynomialFeatures()), ('regression', LinearRegression())])

In [ ]:
poly_grid = GridSearchCV(PolynomialRegression(), param_grid={'polyfeat__degree': range(0, max_degree+1)}, 
                         cv=10, 
                         scoring='neg_mean_squared_error', 
                         verbose=3) 

In [ ]:
poly_grid.fit(train_x, train_y)

In [ ]:
poly_grid.cv_results_

In [ ]:
poly_grid.best_params_

In [ ]:
poly_grid.best_estimator_

In [ ]:
poly_grid.fit(train_x, train_y)

In [ ]:
test_y_pred = poly_grid.predict(test_x)

In [ ]:
get_rmse_r2(test_y, test_y_pred, doPrint=False)

## Statistical Model Selection
After looking at the prediction performance on new data, let's focus on the statistical model selection, which is based on calculations on the training data.

In order to dive deeper into linear Regression, we will use a more specialized library, called `statsmodels`, which we also already imported as `smf`. This library has a class `ols` (for *ordinary least squares*, as we have already seen). We start with a simple linear regression model, aiming to predict `y` based on `x`:

In [ ]:
# define model
lm_grad01 = smf.ols('y ~ x', data=data_train)

# train / "fit" the model
lm_grad01_fitted = lm_grad01.fit()

The `summary` method gives a comprehensive overview over the model and its performance on the training data (see lecture slides for details):

In [ ]:
lm_grad01_fitted.summary()

We continue with a quadratic regression model:

In [ ]:
data_train['x2'] = data_train['x']**2
lm_grad02 = smf.ols('y ~ x + x2 ', data=data_train)
lm_grad02_fitted = lm_grad02.fit()

lm_grad02_fitted.summary()

The cubic regression model:

In [ ]:
data_train['x3'] = data_train['x']**3
lm_grad03 = smf.ols('y ~ x + x2 + x3', data=data_train)
lm_grad03_fitted = lm_grad03.fit()

In [ ]:
lm_grad03_fitted.summary()

**Comment**: We see that as we increase the polynomial degree from 1 to 3, all parameters are significant, and the performance (measured e.g. in the R-squared score) increases.

Now, let's see what happens if we define a polynomial regression model of degree 4:

In [ ]:
data_train['x4'] = data_train['x']**4
lm_grad04 = smf.ols('y ~ x + x2 + x3 + x4', data=data_train)
lm_grad04_fitted = lm_grad04.fit()

lm_grad04_fitted.summary()

**Comment**: As discussed in the slides, we see that the coefficient of `x4` is not significant. We will thus omit it, which gets us back to the polynomial regression up to degree 3.

Hence, based on the statistical analysis of only the training data, the ordinary least squares fitting was able to identify the correct degree of the polynomial, and detected that the degree-4 contribution is not improving for the performance.

## Regularization
As a further way to do model selection, we look at regularization. We will use regularisation to determine the best degree for the polynomial regression.

First we have to scale the predictor variables. We do so using the `StandardScaler` from `sklearn.preprocessing`:

In [ ]:
poly20_scaler = StandardScaler()
poly20_scaler = poly20_scaler.fit(train_x_poly20)
train_x_poly20_std = poly20_scaler.transform(train_x_poly20)

We start with a (randomly chosen) hyperparameter value of 0.1 (note that the hyperparameter is called `alpha` in scikit-learn, unlike in most of the literature):

In [ ]:
lasso_model = Lasso(0.1)

In [ ]:
lasso_model.fit(train_x_poly20_std, train_y)

We can get the coefficients. Note that we are doing multiple linear regression, i.e. we have a series of predictor variables - they are increasing powers of a single predictor variable, but the model does not know (nor need to know) about this. The first coefficient is the coefficient to the constant term $x^0$, then follows the coefficient to $x^1$, etc., until $x^20$:

In [ ]:
lasso_model.coef_

We see that with this weight for the penalty term, we have several non-zero coefficients:

In [ ]:
np.where( np.abs(lasso_model.coef_) > 0)

The coefficients of $x^1$, $x^2$, $x^3$, $x^6$ and $x^19$ are all non-zero.

Let's try with a higher weight for the penalty term:

In [ ]:
lasso_model = Lasso(alpha=0.5, max_iter=100000)
lasso_model.fit(train_x_poly20_std, train_y)
lasso_model.coef_

In [ ]:
np.where( np.abs(lasso_model.coef_) > 0)

Now, the lasso has indeed found the features that were used to generate the data.

## Grid Search for LASSO Parameter

In [ ]:
model = Lasso(random_state = 0)

In [ ]:
alpha_range = [0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10]

In [ ]:
grid_search_r2 = GridSearchCV(model, param_grid={'alpha':alpha_range}, scoring ='r2', cv=5)
grid_search_r2.fit(train_x_poly20_std, train_y)

In [ ]:
print('Scoring R2')
print('Best R2 score   : ', grid_search_r2.best_score_)
print('Best parameters : ', grid_search_r2.best_params_)

In [ ]:
lasso_model_r2 = Lasso(alpha = 2.0)
lasso_model_r2.fit(train_x_poly20_std, train_y)
lasso_model_r2.coef_

In [ ]:
grid_search_mse = GridSearchCV(model, param_grid={'alpha':alpha_range}, scoring ='neg_mean_squared_error', cv=5)
grid_search_mse.fit(train_x_poly20_std, train_y)

In [ ]:
print('Scoring MSE')
print('Best MSE score   : ', -grid_search_mse.best_score_)
print('Best parameters  : ',  grid_search_mse.best_params_)

In [ ]:
lasso_model_mse = Lasso(alpha = 0.5)
lasso_model_mse.fit(train_x_poly20_std, train_y)
lasso_model_mse.coef_

### Impact of the Tuning Parameter
Below we illustrate the impact of the tuning parameter $\alpha$. This is for illustration only, you don't need to understand the code in detail.

In [ ]:
alphas_lasso, coefs_lasso, _ = lasso_path(train_x_poly20_std, train_y, eps=5e-3)

plt.figure(figsize = [10, 7])
for (degree, coef_l) in enumerate(coefs_lasso.squeeze()):
    l1 = plt.semilogx(alphas_lasso, coef_l, label=r'Coefficient of $x^{' + str(degree) + '}$')

plt.xlabel(r'Tuning Parameter $\alpha$')
plt.xticks(ticks=[0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10], labels=[0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10])
plt.ylabel('Coefficient Value')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Lasso')
plt.grid()
plt.show()